# Загрузки и установка библиотек

In [1]:
#@title 1. Keep this tab alive to prevent Colab from disconnecting you { display-mode: "form" }

#@markdown Press play on the music player that will appear below:
%%html
<audio src="https://oobabooga.github.io/silence.m4a" controls>

In [2]:
import requests

def load_file_content(url: str):
    response = requests.get(url) # Получение документа по url.
    response.raise_for_status()  # Проверка ответа и если была ошибка - формирование исключения.
    return response.content

def download_file_from_url(url, save_path):
    content= load_file_content(url)
    with open(save_path, 'wb') as f:
        f.write(content)
        print(f"File downloaded and saved as {save_path}")



In [3]:
http_path = "https://raw.githubusercontent.com/terrainternship/GPT_labsud/main/Petrunin/Streamlit/"
dest_path = "/content/"
func_list = ["requirements.txt", "utils.py"]
for func_name in func_list:
  download_file_from_url(http_path+func_name, func_name)

File downloaded and saved as requirements.txt
File downloaded and saved as utils.py


In [4]:
!pip install -r requirements.txt
from IPython import display
display.clear_output()

# Программа

In [5]:
%%writefile app.py

import streamlit as st
import openai
import tiktoken
from openai import OpenAI
import os

from utils import *

def load_bd():
    if not st.session_state.is_federallab_bd:
        url_promt = "https://raw.githubusercontent.com/terrainternship/GPT_labsud/main/Galina/FLSE_promt"
        url_bd = 'https://github.com/terrainternship/GPT_labsud/raw/main/federallab_bd_index_v2.zip'
        url_question = 'https://github.com/terrainternship/GPT_labsud/raw/main/federallab_bd_question.zip'
        with st.spinner('Загрузка ...'):
            st.session_state.federallab_bd = load_bd_vect(url_bd)
            st.session_state.federallab_question = load_bd_question(
                url_question)
            st.session_state.federallab_promt = load_file(url_promt)
            st.session_state.is_federallab_bd = True

dialog_depth = 3

if "openai_model" not in st.session_state:
    st.session_state["openai_model"] = "gpt-3.5-turbo-1106"

if "messages" not in st.session_state:
    st.session_state.messages = [
        {"role": "assistant", "content": "Наша фирма оказывает услуги по экспертизе и оценке. Чем могу я Вам помочь?"}]

if 'is_federallab_bd' not in st.session_state:
    st.session_state.is_federallab_bd = False

with st.sidebar:
    openai_api_key = st.text_input(
        "OpenAI API Key", key="chatbot_api_key", type="password")
    os.environ["OPENAI_API_KEY"] = openai_api_key
    if st.button("Загрузка Базы знаний...", use_container_width=True):
        if not openai_api_key:
            st.info("Введите ваш OpenAI API key для продолжения.")
            st.stop()
        load_bd()
    if st.session_state.is_federallab_bd:
        st.caption("База знаний загружена.")
    else:
        st.caption("База знаний НЕ загружена.")
    st.divider()
    if st.button("Новый диалог", use_container_width=True):
        st.session_state.messages = [
            {"role": "assistant", "content": "Наша фирма оказывает услуги по экспертизе и оценке. Чем могу я Вам помочь?"}]
    st.divider()

clientOpenAI = OpenAI(api_key=openai_api_key)

with st.container():
    st.header("Нейро ассистент ФЛСЭ")
    stick_it_good()
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])
if query := st.chat_input("Введите свой вопрос."):
    if not openai_api_key:
        st.info("Введите ваш OpenAI API key для продолжения.")
        st.stop()
    if not st.session_state.is_federallab_bd:
        load_bd()
    with st.chat_message("user"):
        message_user = st.empty()
        message_user.markdown(query)
    with st.spinner('Думаю ...'):
        # Проверка вопроса на наличие в базе вопросов.
        is_query, bd_responce = find_query(
            st.session_state.federallab_question, query)
        if not is_query:
            # Формирование по диалогу уточняющего вопроса.
            if len(st.session_state.messages) > 1:
                refined_query = query_refiner(
                    client=clientOpenAI,
                    model=st.session_state["openai_model"],
                    conversation=conversation_string(
                        st.session_state.messages, dialog_depth),
                    query=query)
            else:
                refined_query = query
            message_user.markdown(f"{query} ({refined_query})")
            st.session_state.messages.append(
                {"role": "user", "content": f"{query} ({refined_query})"})
            # Возвращение релевантных документов по запросу.
            content = bd_retrever(
                st.session_state.federallab_bd, refined_query)
            with st.chat_message("assistant"):
                message_placeholder = st.empty()
                full_response = ""
                for response in clientOpenAI.chat.completions.create(
                    model=st.session_state["openai_model"],
                    messages=[
                        {"role": "system", "content": st.session_state.federallab_promt},
                        {"role": "user",   "content": f"Документ с информацией для ответа пользователю:\n {content}.\nВопрос клиента: {refined_query}"}
                    ],
                    temperature=0,
                    stream=True,
                ):
                    full_response += (response.choices[0].delta.content or "")
                    message_placeholder.markdown(full_response + "▌")
                message_placeholder.markdown(full_response)
            st.session_state.messages.append(
                {"role": "assistant", "content": full_response})
        else:
            message_user.markdown(f"{query} (ответ из базы вопросов)")
            st.chat_message("assistant").markdown(bd_responce)

            st.session_state.messages.append(
                {"role": "user", "content": f"{query} (ответ из базы вопросов)"})
            st.session_state.messages.append(
                {"role": "assistant", "content": bd_responce})


Writing app.py


# Запуск WEB-приложения через тунель.

In [7]:
!streamlit run app.py & npx localtunnel --port 8501

[##................] - fetchMetadata: sill resolveWithNewModule yargs-parser@20


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://104.196.158.131:8501

npx: installed 22 in 2.853s
your url is: https://soft-cases-add.loca.lt
/usr/local/lib/python3.10/dist-packages/langchain/vectorstores/__init__.py:35: LangChainDeprecationWarning: Importing vector stores from langchain is deprecated. Importing from langchain will no longer be supported as of langchain==0.2.0. Please import from langchain-community instead:

`from langchain_community.vectorstores import FAISS`.

To install langchain-community run `pip install -U langchain-community`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.OpenAIEmbeddings instead.
  warn_deprecated(
  Sto

1. Запускаем ячейку с кодом - !streamlit run app.py & npx localtunnel --port 8501

In [ ]:
#Порядок запуска. Запустите ячейку откроется картинка.

%%html
<iframe src="https://drive.google.com/file/d/1KbsjKtHAiC3URPoOEIrjshCZjL6IgrT0/preview" width="640" height="480" allow="autoplay"></iframe>